In [ ]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from src.Mlflow_Project.utils.utility import FileOperations
import yaml

class BasePipeline:
    def __init__(self, model_path, config_path):
        self.model = None
        self.config_path = config_path
        self.load_model(model_path)

    def load_model(self, model_path):
        try:
            self.model = joblib.load(model_path)
        except FileNotFoundError:
            print(f"Model file not found at {model_path}")
            self.model = None
        except Exception as e:
            print(f"Error loading model: {e}")
            self.model = None

    def predict(self, data):
        if self.model is not None:
            return self.model.predict(data)
        else:
            return None

class ScalingPipeline(BasePipeline):
    def __init__(self, model_path, config_path):
        super().__init__(model_path, config_path)
        self.scaler = None
        self.load_scaler()

    def load_scaler(self):
        try:
            config = FileOperations.read_yaml(self.config_path)
            scaler_path = Path(config["data_transformation"]["scaler_name"])
            self.scaler = joblib.load(scaler_path)
        except (FileNotFoundError, KeyError):
            print(f"Scaler file not found or config error.")
            self.scaler = None
        except Exception as e:
            print(f"Error loading scaler: {e}")
            self.scaler = None

    def scale(self, data):
        if self.scaler is not None:
            return self.scaler.transform(data)
        else:
            return None

class PredictionPipeline(BasePipeline):
    def __init__(self, model_path, config_path):
        super().__init__(model_path, config_path)

def main():
    config_path = "path/to/your/config.yaml"  # Replace with the actual path to your YAML file
    model_path = "path/to/your/model.joblib"  # Replace with the actual path to your model

    scaling_pipeline = ScalingPipeline(model_path, config_path)
    prediction_pipeline = PredictionPipeline(model_path, config_path)

    # Load your data here
    # data = ...

    scaled_data = scaling_pipeline.scale(data)
    predictions = prediction_pipeline.predict(scaled_data)

if __name__ == "__main__":
    main()


In [ ]:
from flask import Flask, render_template, request
import os
import numpy as np
import pandas as pd
from src.Mlflow_Project.pipeline.prediction import ScalingPipeline, PredictionPipeline

app = Flask(__name__)

# ... Your existing routes and code ...

@app.route('/predict', methods=['POST', 'GET'])
def index():
    if request.method == 'POST':
        try:
            # Reading the inputs given by the user
            engine = int(request.form['engine'])
            cycle = int(request.form['cycle'])
            setting_1 = float(request.form['setting_1'])
            # ... continue reading other input fields ...

            # Convert all input data to a list, then a numpy array
            data = [engine, cycle, setting_1, setting_2, LPC_outlet_temperature_degR, LPT_outlet_temperature_degR, bypass_duct_pressure_psia, HPC_outlet_pressure_psia, Physical_core_speed_rpm, HPC_outlet_Static_pressure_psia, Ratio_of_fuel_flow_to_Ps30_pps_psia, Bypass_Ratio, Bleed_Enthalpy, High_pressure_turbines_Cool_air_flow, Low_pressure_turbines_Cool_air_flow]
            data = np.array(data).reshape(1, -1)

            # Create an instance of ScalingPipeline
            scl = ScalingPipeline(model_path="path/to/your/model.joblib", config_path="path/to/your/config.yaml")

            # Scale the data
            data_scaled = scl.scale(data)

            # Create an instance of PredictionPipeline
            obj = PredictionPipeline(model_path="path/to/your/model.joblib", config_path="path/to/your/config.yaml")

            # Predict using the scaled data
            predict = obj.predict(data_scaled)

            return render_template('results.html', prediction=str(predict))
        except Exception as e:
            print("The Exception message is: ", e)
            return "Something is wrong"
    else:
        return render_template('index.html')

# ... The rest of your existing code ...

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=6060, debug=True)


In [ ]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from src.Mlflow_Project.utils.utility import FileOperations
import yaml

class BasePipeline:
    def __init__(self, model_path, config_path):
        self.model = None
        self.config_path = config_path
        self.load_model(model_path)

    def load_model(self, model_path):
        try:
            self.model = joblib.load(model_path)
        except FileNotFoundError:
            print(f"Model file not found at {model_path}")
            self.model = None
        except Exception as e:
            print(f"Error loading model: {e}")
            self.model = None

    def predict(self, data):
        if self.model is not None:
            return self.model.predict(data)
        else:
            return None

class ScalingPipeline(BasePipeline):
    def __init__(self, model_path, config_path):
        super().__init__(model_path, config_path)
        self.scaler = None
        self.load_scaler()

    def load_scaler(self):
        try:
            with open(self.config_path, 'r') as config_file:
                config = yaml.safe_load(config_file)
                scaler_path = Path(config["data_transformation"]["scaler_name"])
                self.scaler = joblib.load(scaler_path)
        except (FileNotFoundError, KeyError):
            print(f"Scaler file not found or config error.")
            self.scaler = None
        except Exception as e:
            print(f"Error loading scaler: {e}")
            self.scaler = None

    def scale(self, data):
        if self.scaler is not None:
            return self.scaler.transform(data)
        else:
            return None

class PredictionPipeline(BasePipeline):
    def __init__(self, model_path, config_path):
        super().__init__(model_path, config_path)

def main():
    config_path = "path/to/your/config.yaml"  # Replace with the actual path to your YAML file
    model_path = "path/to/your/model.joblib"  # Replace with the actual path to your model

    scaling_pipeline = ScalingPipeline(model_path, config_path)
    prediction_pipeline = PredictionPipeline(model_path, config_path)

    # Load your data here
    # data = ...

    scaled_data = scaling_pipeline.scale(data)
    predictions = prediction_pipeline.predict(scaled_data)

if __name__ == "__main__":
    main()
